In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_avccmdm342',
    proc_date,
    'temp_sap_avccmdm342'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm342 (
hash_id BIGINT,
bbynr STRING,
recordmode STRING,
crdate DATE,
crname STRING,
chdate DATE,
chname STRING,
categ STRING,
type STRING,
point STRING,
maxcount STRING,
bprof STRING,
datab DATE,
datbi DATE,
aktnr STRING,
buycon STRING,
getcon STRING,
crappl STRING,
crdocn STRING,
status STRING,
bbycurh STRING,
offer_id STRING,
newmain STRING,
limitnr FLOAT,
buy_prqval_min FLOAT,
buy_prqcur STRING,
synch_to_promo STRING,
zzudtime STRING,
proc_date DATE,
start_date DATE,
end_date DATE,
file_creation_ts TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW sap_avccmdm342 AS
WITH temp AS (
    SELECT * EXCEPT(rn) FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY bbynr, DATE(proc_date) ORDER BY proc_date) rn from (
        SELECT
            IF(bbynr = '', NULL, LTRIM('0', bbynr)) AS bbynr,
            IF(recordmode = '', NULL, recordmode) AS recordmode,
            TO_DATE(IF(crdate IN ('', '00000000'), NULL, crdate), 'yyyyMMdd') AS crdate,
            IF(crname = '', NULL, crname) AS crname,
            TO_DATE(IF(chdate IN ('', '00000000'), NULL, chdate), 'yyyyMMdd') AS chdate,
            IF(chname = '', NULL, chname) AS chname,
            IF(categ = '', NULL, categ) AS categ,
            IF(type = '', NULL, type) AS type,
            IF(point = '', NULL, point) AS point,
            IF(maxcount = '', NULL, maxcount) AS maxcount,
            IF(bprof = '', NULL, bprof) AS bprof,
            TO_DATE(IF(datab IN ('', '00000000'), NULL, datab), 'yyyyMMdd') AS datab,
            TO_DATE(IF(datbi IN ('', '00000000'), NULL, datbi), 'yyyyMMdd') AS datbi,
            IF(aktnr = '', NULL, aktnr) AS aktnr,
            IF(buycon = '', NULL, buycon) AS buycon,
            IF(getcon = '', NULL, getcon) AS getcon,
            IF(crappl = '', NULL, crappl) AS crappl,
            IF(crdocn = '', NULL, crdocn) AS crdocn,
            IF(status = '', NULL, status) AS status,
            IF(bbycurh = '', NULL, bbycurh) AS bbycurh,
            IF(offer_id = '', NULL, offer_id) AS offer_id,
            IF(newmain = '', NULL, newmain) AS newmain,
            limitnr,
            buy_prqval_min,
            IF(buy_prqcur = '', NULL, buy_prqcur) AS buy_prqcur,
            IF(synch_to_promo = '', NULL, synch_to_promo) AS synch_to_promo,
            IF(zzudtime = '', NULL, zzudtime) AS zzudtime,
            proc_date,
            file_creation_ts
        FROM temp_sap_avccmdm342
        ) a
    ) a WHERE rn = 1
),
main AS (
    SELECT farm_fingerprint(CONCAT(
        IFNULL(CAST(bbynr AS STRING), ""),
        IFNULL(CAST(recordmode AS STRING), ""),
        IFNULL(CAST(crdate AS STRING), ""),
        IFNULL(CAST(crname AS STRING), ""),
        IFNULL(CAST(chdate AS STRING), ""),
        IFNULL(CAST(chname AS STRING), ""),
        IFNULL(CAST(categ AS STRING), ""),
        IFNULL(CAST(type AS STRING), ""),
        IFNULL(CAST(point AS STRING), ""),
        IFNULL(CAST(maxcount AS STRING), ""),
        IFNULL(CAST(bprof AS STRING), ""),
        IFNULL(CAST(datab AS STRING), ""),
        IFNULL(CAST(datbi AS STRING), ""),
        IFNULL(CAST(aktnr AS STRING), ""),
        IFNULL(CAST(buycon AS STRING), ""),
        IFNULL(CAST(getcon AS STRING), ""),
        IFNULL(CAST(crappl AS STRING), ""),
        IFNULL(CAST(crdocn AS STRING), ""),
        IFNULL(CAST(status AS STRING), ""),
        IFNULL(CAST(bbycurh AS STRING), ""),
        IFNULL(CAST(offer_id AS STRING), ""),
        IFNULL(CAST(newmain AS STRING), ""),
        IFNULL(CASE WHEN CAST(limitnr AS DOUBLE) = CAST(CAST(limitnr AS DOUBLE) AS BIGINT) THEN CAST(CAST(limitnr AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(limitnr AS DECIMAL(38,10)))) END, ""),
        IFNULL(CASE WHEN CAST(buy_prqval_min AS DOUBLE) = CAST(CAST(buy_prqval_min AS DOUBLE) AS BIGINT) THEN CAST(CAST(buy_prqval_min AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(buy_prqval_min AS DECIMAL(38,10)))) END, ""),
        IFNULL(CAST(buy_prqcur AS STRING), ""),
        IFNULL(CAST(synch_to_promo AS STRING), ""),
        IFNULL(CAST(zzudtime AS STRING), "")
    )) AS hash_id, *, DATE(proc_date) AS start_date
    FROM temp
),
main_2 AS (
    SELECT *, LAG(hash_id, 1) OVER (PARTITION BY bbynr ORDER BY proc_date) AS hash_id_prev 
    FROM main
),
main_3 AS (
    SELECT * EXCEPT(hash_id_prev) 
    FROM main_2 
    WHERE (hash_id != hash_id_prev OR hash_id_prev IS NULL)
),
main_4 AS (
    SELECT *, LEAD(DATE(proc_date)) OVER (PARTITION BY bbynr ORDER BY proc_date ASC) AS end_date 
    FROM main_3
)
SELECT * EXCEPT(end_date), IFNULL(date_add(end_date, -1), '2400-01-01') AS end_date 
FROM main_4
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm342 inf_tbl
USING (
    SELECT bbynr, hash_id, start_date FROM sap_avccmdm342
        WHERE end_date = DATE '2400-01-01'
) tmp_tbl
ON inf_tbl.bbynr = tmp_tbl.bbynr AND inf_tbl.hash_id != tmp_tbl.hash_id AND inf_tbl.end_date = DATE '2400-01-01'
WHEN MATCHED THEN UPDATE SET inf_tbl.end_date = DATE_ADD(tmp_tbl.start_date,-1)
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm342 inf_tbl
	USING sap_avccmdm342 tmp_tbl
	ON inf_tbl.hash_id = tmp_tbl.hash_id  AND inf_tbl.end_date = DATE '2400-01-01'
	WHEN NOT MATCHED THEN INSERT *
 """)

In [0]:
spark.sql("DROP VIEW IF EXISTS temp_sap_avccmdm342")
spark.sql("DROP VIEW IF EXISTS sap_avccmdm342")